In [5]:
# filepath: c:\Users\mahme\trading_bot\notebooks\train_lstm.ipynb
import os
import sys
sys.path.append(os.path.abspath(".."))

In [6]:
import pandas as pd
from src.data_fetch import connect_mt5, fetch_symbol_data
from src.features import (add_RSI_EMA, add_ATR, add_candlestick_patterns,
                          add_VSA_signals, add_order_blocks, create_sequences)
from src.model_lstm import train_model

# 1. Fetch or load CSV
df = pd.read_csv("data/XAUUSD_M5.csv", parse_dates=["time"])

# 2. Compute features
df = add_RSI_EMA(df, rsi_period=14, ema_periods=[20, 50])
df = add_ATR(df, atr_period=14)
df = add_candlestick_patterns(df)
df = add_VSA_signals(df)
df = add_order_blocks(df)

# 3. Generate a direction label
df["Direction"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
df.dropna(inplace=True)

# 4. Choose feature columns for the model
feature_cols = [
    "Close", "RSI", "EMA_20", "EMA_50", "ATR",
    "HAMMER", "ENGULFING", "DOJI",
    "Climactic_Spread", "No_Demand", "No_Supply",
    "Bullish_OB", "Bearish_OB"
]
label_col = "Direction"

# 5. Split into train/val (e.g., 80/20 chronological split)
train_size = int(len(df) * 0.8)
df_train = df.iloc[:train_size]
df_val = df.iloc[train_size:]

# 6. Create sequences
lookback = 60  # e.g., last 60 M5 bars (~5 hours)
X_train, y_train, scaler = create_sequences(df_train, feature_cols, label_col, lookback)
X_val, y_val, _ = create_sequences(df_val, feature_cols, label_col, lookback)

# 7. Train
input_shape = (X_train.shape[1], X_train.shape[2])
model, history = train_model(X_train, y_train, X_val, y_val, input_shape,
                             model_save_path="models/XAUUSD_M5_lstm.h5",
                             epochs=30, batch_size=64)


ModuleNotFoundError: No module named 'pandas'